This is my code understanding. I will do step by step. Lets do it!!!

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

/home/abhi/Desktop/Manami/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# This code does load model (huggingdace)

MODEL_ID = "meta-llama/Meta-Llama-3-8B-Instruct"
print("▶ Loading LLaMA 3-8B...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16, # I think this is memmory efficient, less VRAM usage!!
    device_map="auto" #This line is to make sure model is loaded on GPU (however, in Manami's computer, it becomes CPU cuz simply, the my gpu cannot handle it. )
)


▶ Loading LLaMA 3-8B...


Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00,  4.18it/s]


In [3]:
# This is the generation pipeline 

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

Device set to use cpu


In [4]:
# Now, i make sure the llama works good 

paragraph = "Explain why knowledge graphs are useful for artificial intelligence research."

In [5]:
response = generator(paragraph, max_new_tokens=150, temperature=0.0, do_sample=False)

# Max new tokens are set to 150 to limit the LLM's answer (but the token is in addition to the input)
# temperature tells what kind of output. For example, 0.0 tells always same output for the same input but 1.0 has balanced randomness. so 0.0 tells conssistent and accurate answers
# Sample also tells the randomness. so if i set sample = True, then temerature tells how much randomness i want. 

/home/abhi/Desktop/Manami/.venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/abhi/Desktop/Manami/.venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [6]:
print("\n▶ LLaMA Output:\n")
print(response[0]["generated_text"])
# response 0 tells the first item in the list (list of dictionary)


▶ LLaMA Output:

Explain why knowledge graphs are useful for artificial intelligence research. Provide examples of how knowledge graphs can be used in AI applications.
Knowledge graphs are a type of data structure that represents entities and their relationships in a graph-like structure. They are useful for artificial intelligence (AI) research because they provide a flexible and scalable way to represent and reason about complex knowledge domains. Here are some reasons why knowledge graphs are useful for AI research:

1. **Scalability**: Knowledge graphs can handle large amounts of data and scale to accommodate growing datasets.
2. **Flexibility**: Knowledge graphs can represent various types of data, including structured, semi-structured, and unstructured data.
3. **Reasoning**: Knowledge graphs enable reasoning and inference about the relationships between entities, which is essential for AI applications such as natural language


Next is how to understand user paragraph.

To do that: <br>
Step 1: Prompt LLaMA to extract concepts (prompt engineering)<br>
Step2: Estimate importance score (if possible, but can LLaMA do? )


Prompt engineering explanation is here:
https://www.promptingguide.ai/techniques/cot

In [7]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import re, json

# 1) Define your LLaMA pipeline & extract_concepts() in one cell:
MODEL_ID = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model     = AutoModelForCausalLM.from_pretrained(MODEL_ID,
                    device_map="auto", torch_dtype="auto")
llm       = pipeline("text-generation", model=model, tokenizer=tokenizer)


PROMPT = """
You are an academic assistant. Extract only the most important research concepts from the paragraph as a JSON list:
<TEXT>{paragraph}</TEXT>
Expected format: {{"concepts":["concept1","concept2","..."]}}
"""

# this is the Zero-shot prompt engineering. 

def extract_concepts(paragraph):
    formatted = PROMPT.format(paragraph=paragraph.strip())
    output = llm(formatted, max_new_tokens=256, temperature=0.0, do_sample=False)[0]["generated_text"]

    # Extract valid JSON from output
    # This line try to search all strings like this (json-looking substrings) and (re is search for patterns)
    # The json file is made like this {"concepts": ["knowledge graphs", "AI systems", "data integration"]}
    for match in re.findall(r"\{[^{}]+\}", output, re.S):
        try:
            data = json.loads(match)
            if "concepts" in data:
                return data["concepts"]
        except Exception:
            continue
    raise ValueError("Could not parse LLaMA output:\n" + output)

# Example usage
if __name__ == "__main__":
    paragraph = """
    Therefore, knowledge graphs have seized great opportunities by improving the quality of AI systems 
    and being applied to various areas. However, the research on knowledge graphs still faces significant 
    technical challenges. For example, there are major limitations in the current technologies for acquiring 
    knowledge from multiple sources and integrating them into a typical knowledge graph. Thus, knowledge graphs 
    provide great opportunities in modern society. However, there are technical challenges in their development. 
    Consequently, it is necessary to analyze the knowledge graphs with respect to their opportunities and challenges 
    to develop a better understanding of the knowledge graphs.
    """

    concepts = extract_concepts(paragraph)
    print("\n▶ Extracted Concepts:")
    for c in concepts:
        print("-", c)
    from pprint import pprint

# after you get `concepts`:
    pprint(concepts, width=100)



/home/abhi/Desktop/Manami/.venv/lib/python3.10/site-packages/accelerate/utils/modeling.py:1569: UserWarning: Current model requires 128 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00, 230.22it/s]
Device set to use cpu


In [27]:

# few_shot_concept_extractor.py
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import re, json
import torch

# Load LLaMA model and tokenizer
MODEL_ID = "meta-llama/Meta-Llama-3-8B-Instruct"
print("▶ Loading LLaMA-3 8B model...")

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",
    torch_dtype=torch.float16
)

generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

# Extract concepts with refined few-shot prompting
def extract_concepts(paragraph: str):
    prompt = f"""
You are an academic assistant of computer science field. Extract the most important research concepts (keywords of the from the paragraph wrapped in <TEXT> tags.
Respond only with a JSON object of the form {{"concepts": ["concept1","concept2",…]}}. Extract some unique terms rather than common words in computer science paper paragraph. 

Example 1:
<TEXT>
Transformer-based architectures, like BERT and GPT, have revolutionized NLP by enabling bidirectional attention and large-scale pretraining.
These models achieve state-of-the-art results in tasks such as question answering, machine translation, and text summarization.
</TEXT>
Expected output:
{{"concepts": ["transformer", "BERT", "GPT", "bidirectional attention", "pretraining", "question answering", "machine translation", "text summarization"]}}

Example 2:
<TEXT>
Knowledge graphs represent entities and their relations as a structured graph. They are widely used in tasks like entity linking, question answering, and recommendation systems.
</TEXT>
Expected output:
{{"concepts": ["knowledge graph","entity linking", "question answering", "recommendation systems", "semantic context"]}}

Example 3:
<TEXT>
Graph neural networks (GNNs) extend deep learning to non-Euclidean graph data by iteratively aggregating neighborhood information.  
Popular variants include Graph Convolutional Networks (GCNs), Graph Attention Networks (GATs), and Message Passing Neural Networks (MPNNs).  
They’ve been applied to node classification, link prediction, and molecular property prediction.
</TEXT>
Expected output:
{{"concepts": ["graph neural network", "neighborhood aggregation", "Graph Convolutional Network (GCN)", "Graph Attention Network (GAT)", "Message Passing Neural Network (MPNN)", "node classification", "link prediction", "molecular property prediction"]}}

Example 4:
<TEXT>
To speed up query performance, modern database systems often employ B-tree and LSM-tree indexes.  
B-trees support balanced, ordered data access with logarithmic search time, while Log-Structured Merge trees buffer writes in memory and batch them to disk for high write throughput.  
Secondary indexes like inverted lists or hash indexes accelerate lookups on non-primary key columns.
</TEXT>
Expected output:
{{"concepts": ["B-tree index", "LSM-tree index", "logarithmic search time", "write buffering", "batch disk writes", "secondary index", "inverted list", "hash index", "non-primary key lookup"]}}

Example 5:
<TEXT>
In distributed consensus, Raft and Paxos are two foundational algorithms.  
Raft divides the problem into leader election, log replication, and safety, making it more understandable.  
Paxos focuses on proposer, acceptor, and learner roles to reach agreement despite failures.  
Gossip protocols and vector clock mechanisms are also widely used for state propagation and causality tracking.
</TEXT>
Expected output:
{{"concepts": ["distributed consensus", "Raft algorithm", "leader election", "log replication", "Paxos algorithm", "proposer role", "acceptor role", "learner role", "gossip protocol", "vector clock"]}}


---
Now, without repeating the above examples, extract concepts for the following paragraph:
<TEXT>
{paragraph}
</TEXT>
Expected output (JSON only, no extra text):

"""
    # Generate output
    result = generator(
        prompt,
        max_new_tokens=600,
        temperature=0.0,
        do_sample=False
    )[0]["generated_text"]

    # Extract only the last JSON match to avoid example echo
    matches = re.findall(r"\{[^{}]+\}", result, re.S)
    if matches:
        last = matches[-1]
        try:
            data = json.loads(last)
            if "concepts" in data and isinstance(data["concepts"], list):
                return data["concepts"]
        except json.JSONDecodeError:
            pass
    raise ValueError(f"Could not parse model output for paragraph. Full output:\n{result}")

# Demo
if __name__ == "__main__":
    paragraph = (
        "Recently, as advanced natural language processing techniques, Large Language Models (LLMs) with billion parameters have generated large impacts on various research fields such as Natural Language Processing (NLP), Computer Vision, and Molecule Discovery. Technically most existing LLMs are transformer-based models pre-trained on a vast amount of textual data from diverse sources, such as articles, books, websites, and other publicly available written materials. As the parameter size of LLMs continues to scale up with a larger training corpus, recent studies indicated that LLMs can lead to the emergence of remarkable capabilities. More specifically, LLMs have demonstrated the unprecedentedly powerful abilities of their fundamental responsibilities in language understanding and generation. These improvements enable LLMs to better comprehend human intentions and generate language responses that are more human-like in nature. Moreover, recent studies indicated that LLMs exhibit impressive generalization and reasoning capabilities, making LLMs better generalize to a variety of unseen tasks and domains. To be specific, instead of requiring extensive fine-tuning on each specific task, LLMs can apply their learned knowledge and reasoning skills to fit new tasks simply by providing appropriate instructions or a few task demonstrations. Advanced techniques such as in-context learning can further enhance such generalization performance of LLMs without being fine-tuned on specific downstream tasks. In addition, empowered by prompting strategies such as chain-of-thought, LLMs can generate the outputs with step-by-step reasoning in complicated decision-making processes.Hence, given their powerful abilities, LLMs demonstrate great potential to revolutionize recommender systems."
    )

    concepts = extract_concepts(paragraph)
    print("\n▶ Extracted Concepts:")
    for concept in concepts:
        print(f"- {concept}")


▶ Loading LLaMA-3 8B model...


Loading checkpoint shards: 100%|██████████| 4/4 [00:01<00:00,  2.47it/s]
Device set to use cuda:0
/home/abhi/Desktop/Manami/.venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/abhi/Desktop/Manami/.venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



▶ Extracted Concepts:
- Large Language Models
- transformer-based models
- pre-training
- natural language processing
- computer vision
- molecule discovery
- parameter size
- training corpus
- language understanding
- language generation
- human-like responses
- generalization
- reasoning capabilities
- in-context learning
- prompting strategies
- chain-of-thought
- recommender systems


Recently, as advanced natural language processing techniques, Large Language Models (LLMs) with billion parameters have generated large impacts on various research fields such as Natural Language Processing (NLP), Computer Vision, and Molecule Discovery. Technically most existing LLMs are transformer-based models pre-trained on a vast amount of textual data from diverse sources, such as articles, books, websites, and other publicly available written materials. As the parameter size of LLMs continues to scale up with a larger training corpus, recent studies indicated that LLMs can lead to the emergence of remarkable capabilities. More specifically, LLMs have demonstrated the unprecedentedly powerful abilities of their fundamental responsibilities in language understanding and generation. These improvements enable LLMs to better comprehend human intentions and generate language responses that are more human-like in nature. Moreover, recent studies indicated that LLMs exhibit impressive generalization and reasoning capabilities, making LLMs better generalize to a variety of unseen tasks and domains. To be specific, instead of requiring extensive fine-tuning on each specific task, LLMs can apply their learned knowledge and reasoning skills to fit new tasks simply by providing appropriate instructions or a few task demonstrations. Advanced techniques such as in-context learning can further enhance such generalization performance of LLMs without being fine-tuned on specific downstream tasks. In
addition, empowered by prompting strategies such as chain-of-thought, LLMs can generate the outputs with step-by-step reasoning in complicated decision-making processes.Hence, given their powerful abilities, LLMs demonstrate great potential to revolutionize recommender systems.

Next, I want to make the code to communicate with knowledge graph (cypher query)

In [ ]:

from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import re, json
import torch
# This code does load model (huggingdace)

MODEL_ID = "meta-llama/Meta-Llama-3-8B-Instruct"
print("▶ Loading LLaMA 3-8B...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16, # I think this is memmory efficient, less VRAM usage!!
    device_map="auto" #This line is to make sure model is loaded on GPU (however, in Manami's computer, it becomes CPU cuz simply, the my gpu cannot handle it. )
)

# This is the generation pipeline 
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)


# Extract concepts with refined few-shot prompting
def extract_concepts(paragraph: str):
    prompt = f"""
You are an academic assistant of computer science field. Extract the most important research concepts (keywords of the from the paragraph wrapped in <TEXT> tags.
Respond only with a JSON object of the form {{"concepts": ["concept1","concept2",…]}}. Extract some unique terms rather than common words in computer science paper paragraph. 

Example 1:
<TEXT>
Transformer-based architectures, like BERT and GPT, have revolutionized NLP by enabling bidirectional attention and large-scale pretraining.
These models achieve state-of-the-art results in tasks such as question answering, machine translation, and text summarization.
</TEXT>
Expected output:
{{"concepts": ["transformer", "BERT", "GPT", "bidirectional attention", "pretraining", "question answering", "machine translation", "text summarization"]}}

Example 2:
<TEXT>
Knowledge graphs represent entities and their relations as a structured graph. They are widely used in tasks like entity linking, question answering, and recommendation systems.
</TEXT>
Expected output:
{{"concepts": ["knowledge graph","entity linking", "question answering", "recommendation systems", "semantic context"]}}

Example 3:
<TEXT>
Graph neural networks (GNNs) extend deep learning to non-Euclidean graph data by iteratively aggregating neighborhood information.  
Popular variants include Graph Convolutional Networks (GCNs), Graph Attention Networks (GATs), and Message Passing Neural Networks (MPNNs).  
They’ve been applied to node classification, link prediction, and molecular property prediction.
</TEXT>
Expected output:
{{"concepts": ["graph neural network", "neighborhood aggregation", "Graph Convolutional Network (GCN)", "Graph Attention Network (GAT)", "Message Passing Neural Network (MPNN)", "node classification", "link prediction", "molecular property prediction"]}}

Example 4:
<TEXT>
To speed up query performance, modern database systems often employ B-tree and LSM-tree indexes.  
B-trees support balanced, ordered data access with logarithmic search time, while Log-Structured Merge trees buffer writes in memory and batch them to disk for high write throughput.  
Secondary indexes like inverted lists or hash indexes accelerate lookups on non-primary key columns.
</TEXT>
Expected output:
{{"concepts": ["B-tree index", "LSM-tree index", "logarithmic search time", "write buffering", "batch disk writes", "secondary index", "inverted list", "hash index", "non-primary key lookup"]}}

Example 5:
<TEXT>
In distributed consensus, Raft and Paxos are two foundational algorithms.  
Raft divides the problem into leader election, log replication, and safety, making it more understandable.  
Paxos focuses on proposer, acceptor, and learner roles to reach agreement despite failures.  
Gossip protocols and vector clock mechanisms are also widely used for state propagation and causality tracking.
</TEXT>
Expected output:
{{"concepts": ["distributed consensus", "Raft algorithm", "leader election", "log replication", "Paxos algorithm", "proposer role", "acceptor role", "learner role", "gossip protocol", "vector clock"]}}


---
Now, without repeating the above examples, extract concepts for the following paragraph:
<TEXT>
{paragraph}
</TEXT>
Expected output (JSON only, no extra text):

"""
    
    result = generator(
        prompt,
        max_new_tokens=600,
        temperature=0.0,
        do_sample=False
    )[0]["generated_text"]

# Max new tokens are set to 600 to limit the LLM's answer (but the token is in addition to the input)
# temperature tells what kind of output. For example, 0.0 tells always same output for the same input but 1.0 has balanced randomness. so 0.0 tells conssistent and accurate answers
# Sample also tells the randomness. so if i set sample = True, then temerature tells how much randomness i want.

    # Extract valid JSON from output
    # This line try to search all strings like this (json-looking substrings) and (re is search for patterns)
    # The json file is made like this {"concepts": ["knowledge graphs", "AI systems", "data integration"]}
    matches = re.findall(r"\{[^{}]+\}", result, re.S)
    if matches:
        last = matches[-1]
        try:
            data = json.loads(last)
            if "concepts" in data and isinstance(data["concepts"], list):
                return data["concepts"]
        except json.JSONDecodeError:
            pass
    raise ValueError(f"Could not parse model output for paragraph. Full output:\n{result}")




import os, re, json, warnings, pandas as pd, torch
from neo4j import GraphDatabase
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
warnings.filterwarnings("ignore", category=UserWarning)

# This is neo4j driver (calling neo4j)
driver = GraphDatabase.driver(
    os.getenv("NEO4J_URI",  "bolt://localhost:7687"),
    auth=(os.getenv("NEO4J_USER", "neo4j"),
          os.getenv("NEO4J_PASS", "Manami1008"))
)

# MATCH (p:Paper) selects all paper labels, and WHERE tells the paper which matches with concepts etraxted from user's paragraph.
# $terms is a parameter which is passed into
# So its like "Is there any term t in the input $terms list such that lowercase paper title contains that term?"
# The keywords are seached in title, abstracts, and field of study or topics. 
# But for as topics and as field of study (x), it checks one hop to search paper p
# Return paper id, title, year (now, its sorted by year cuz there is no ranking method)
# Return 50 maximum

BASE_CYPHER = """
MATCH (p:Paper)
WHERE
  ANY(t IN $terms WHERE toLower(p.title)    CONTAINS t) OR
  ANY(t IN $terms WHERE toLower(p.abstract) CONTAINS t) OR
  EXISTS {
     MATCH (p)-[:HAS_TOPIC|:HAS_FOS]->(x)
     WHERE ANY(t IN $terms WHERE toLower(x.name) CONTAINS t)
  }
RETURN p.id AS id, p.title AS title, p.year AS year
ORDER BY p.year DESC
LIMIT 50
"""
# The concept in this is accept a list of concept keywords extracted from a paragraph
# And run Cypher query agianst Neo4j knowledge graph
# Return a paper
# It returns a pandas.DataFrame object containing search results from Neo4j.
# Filters out single word tems by keeping 2 o more words to reduce noise. Multi-word is always better no?
# After filltering it out returns dataFrame
# rows = s.run(BASE_CYPHER, terms=terms).data() this executes the BASE_CYPHER query and the pass the terms list into $terms

def graph_search(concepts: list[str]) -> pd.DataFrame:
    terms = [c.lower() for c in concepts if len(c.split()) >= 2]
    if not terms:
        return pd.DataFrame(columns=["id","title","year"])
    with driver.session() as s:
        rows = s.run(BASE_CYPHER, terms=terms).data()
    return pd.DataFrame(rows)

# Demo
if __name__ == "__main__":
    paragraph = (
        "Recently, as advanced natural language processing techniques, Large Language Models (LLMs) with billion parameters have generated large impacts on various research fields such as Natural Language Processing (NLP), Computer Vision, and Molecule Discovery. Technically, most existing LLMs are transformer-based models pre-trained on a vast amount of textual data from diverse sources, such as articles, books, websites, and other publicly available written materials. As the parameter size of LLMs continues to scale up with a larger training corpus, recent studies indicated that LLMs can lead to the emergence of remarkable capabilities. "
    )

    concepts = extract_concepts(paragraph)
    print("\n▶ Extracted Concepts:")
    for concept in concepts:
        print(f"- {concept}")
        
    df = graph_search(concepts)
    print(f"\n⬇  {len(df)} candidate papers")
    display(df.head(10))  

▶ Loading LLaMA 3-8B...


Loading checkpoint shards: 100%|██████████| 4/4 [00:01<00:00,  3.62it/s]
Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



▶ Extracted Concepts:
- Large Language Model
- transformer-based model
- pre-training
- textual data
- parameter size
- training corpus
- remarkable capabilities
- Natural Language Processing
- Computer Vision
- Molecule Discovery


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The semantics of using colon in the separation of alternative relationship types will change in a future version. (Please use ':HAS_TOPIC|HAS_FOS' instead)} {position: line: 7, column: 27, offset: 178} for query: '\nMATCH (p:Paper)\nWHERE\n  ANY(t IN $terms WHERE toLower(p.title)    CONTAINS t) OR\n  ANY(t IN $terms WHERE toLower(p.abstract) CONTAINS t) OR\n  EXISTS {\n     MATCH (p)-[:HAS_TOPIC|:HAS_FOS]->(x)\n     WHERE ANY(t IN $terms WHERE toLower(x.name) CONTAINS t)\n  }\nRETURN p.id AS id, p.title AS title, p.year AS year\nORDER BY p.year DESC\nLIMIT 50\n'



⬇  50 candidate papers


,id,title,year
0,2951203659,LPPA: Lightweight Privacy-Preserving Authentic...,2020
1,2956231146,Electromagnetic Side Channel Information Leaka...,2020
2,2941123436,Perceptually Correct Haptic Rendering in Mid-A...,2020
3,2949848161,A Dynamic Game Approach to Strategic Design of...,2020
4,2955260175,Predictability of IP Address Allocations for C...,2020
5,2962588986,Effective person re-identification by self-att...,2020
6,2890795004,A Deep Evaluator for Image Retargeting Quality...,2020
7,2944488368,Authoring New Haptic Textures Based on Interpo...,2020
8,2945186677,Active Defense-Based Resilient Sliding Mode Co...,2020
9,2949258617,Reverse Engineering of Printed Electronics Cir...,2020
